In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr  6 01:34:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# install asteroid
!git clone https://github.com/asteroid-team/asteroid
%cd asteroid
!pip install -e .
!pip install asteroid

!pip install pytorch-lightning

fatal: destination path 'asteroid' already exists and is not an empty directory.
/content/asteroid
Obtaining file:///content/asteroid
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: asteroid
    Found existing installation: asteroid 0.5.2
    Uninstalling asteroid-0.5.2:
      Successfully uninstalled asteroid-0.5.2
  Running setup.py develop for asteroid
  Using cached asteroid-0.5.2-py3-none-any.whl (246 kB)


In [4]:
import torch
from torch import optim
from pytorch_lightning import Trainer
from asteroid.models import ConvTasNet
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper
from asteroid.data import LibriMix
from asteroid.engine import System
from torch.utils.data import DataLoader

# to enable cuda, you have to do a manual toggle in the settings
# menu 'runtime->change runtime->hardware accelerator->gpu'
torch.cuda.is_available()

True

In [5]:
# import custom data set from py in google drive
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive')
from custom_dataset_sandbox import CustomMixture 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
csv_path = '/content/drive/MyDrive/output/'
%cd /content/drive/MyDrive/output/
train_loader = DataLoader(CustomMixture(csv_path, "truetrain_metadata.csv"), num_workers=8, batch_size=2)
val_loader = DataLoader(CustomMixture(csv_path, "trueval_metadata.csv"), num_workers=8, batch_size=2)

/content/drive/MyDrive/output


In [7]:
model = ConvTasNet(n_src=2)

# PITLossWrapper works with any loss function.
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

system = System(model, optimizer, loss, train_loader, val_loader)

In [ ]:
trainer = Trainer(max_epochs=20, gpus=1, auto_select_gpus=True, log_every_n_steps=1)
trainer.fit(system)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | model     | ConvTasNet     | 5.1 M 
1 | loss_func | PITLossWrapper | 0     
---------------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.202    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# save model
model_path = '/content/drive/MyDrive/output/model_saves/iteration2_part1.pt'
torch.save(model.state_dict(), model_path)

In [ ]:
# load model

# model1 = ConvTasNet(n_src=2)
# model1.load_state_dict(torch.load(model_path))
# model1.eval()

In [ ]:
# test with specific audio clips

# import librosa
# audiofile, sr_rate = librosa.load('/content/drive/MyDrive/output/mixture/mixtureclip16027.wav') 
# prediction = model1(torch.from_numpy(audiofile))
# prediction1 = prediction

# import soundfile as sf
# sf.write('/content/drive/MyDrive/output/3v_test_wav2.wav', prediction1.detach().numpy()[1], sr_rate)



In [ ]:
#print(prediction.detach().numpy()[0].shape)

In [ ]:
# import torch
# from google.colab import drive
# drive.mount('/content/drive')
# import sys
# sys.path.insert(0,'/content/drive/My Drive')
# torch.cuda.is_available()
# checkpoint = torch.load('/content/drive/MyDrive/output/lightning_logs/version_2/checkpoints/epoch=0-step=11467.ckpt')
# torch.cuda.is_available()

# #print(checkpoint.keys())
# model = ConvTasNet(n_src=2)
# # model.from_pretrained(ConvTasNet, checkpoint['state_dict'])
# model.load_state_dict(checkpoint['state_dict'])